# 单GPU使用简介

查看GPU信息

In [3]:
!nvidia-smi

Mon Jan  8 00:17:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.92                 Driver Version: 545.92       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   37C    P8              12W /  95W |   2168MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## 表达硬件

显示计算设备信息等

In [4]:
import torch as tor
from torch import nn as tor_nn

In [6]:
tor.device('cpu'), tor.cuda.device('cuda:0') # 注意，如果有多个GPU，可用cuda:n来表示访问第几个gpu

(device(type='cpu'), <torch.cuda.device at 0x151c4c5e850>)

In [7]:
tor.cuda.device_count() # 查看有几个可用gpu

1

### 尝试找gpu

请求GPU是否存在的代码

In [9]:
def try_gpu(i = 0):
    """查看是否存在gpu,如果存在,则返回第i个gpu,如果不存在,则返回cpu"""
    if tor.cuda.device_count() >= i + 1:
        return tor.device(f'cuda:{i}')
    else:
        return tor.device('cpu')

def try_all_gpus():
    """返回所有可用gpu,如果没有,则返回cpu"""
    devices = [tor.device(f'cuda:{i}') for i in range(tor.cuda.device_count())]
    if devices:
        return devices
    else:
        return [tor.device('cpu')]

try_gpu(), try_all_gpus()

(device(type='cuda', index=0), [device(type='cuda', index=0)])

### 张量位置

查询现在张量所在设备

In [10]:
x = tor.tensor([1, 2, 3]) # 默认cpu上
x.device

device(type='cpu')

创建一个张量位于GPU上

In [19]:
X = tor.tensor([1.0, 2.0, 3.0], device=try_gpu())
X

tensor([1., 2., 3.], device='cuda:0')

注意：在计算中我们应当保证我们的计算是放在CPU或者GPU上运行的，另外一个点,我们应当保证我们要计算的值是放在同一张gpu的卡上的(因为gpu是一个协处理器，没有权限自主调用，所以我们如果要计算，应当保证其是在一张卡上)

In [17]:
Y = tor.tensor([2, 4, 5], device=try_gpu())
X + Y

tensor([3, 6, 8], device='cuda:0')

!!!cpu向gpu挪数据是很慢的！！！

## 网络模型与GPU

In [21]:
net = tor_nn.Sequential(tor_nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(X) # 注意：gpu嘴巴很叼的，只要float型哦

tensor([-0.9678], device='cuda:0', grad_fn=<AddBackward0>)

确认是在gpu上运行

In [22]:
net[0].weight.data.device

device(type='cuda', index=0)

数据转存

In [29]:
cs = tor.ones(2, 3)
print(cs.device)
cs = cs.cuda() # 数据转换到gpu
print(cs.device)

cpu
cuda:0
